In [1]:
import numpy as np
import random
from joblib import Parallel, delayed

In [2]:
class Knight:
    def __init__(self,x,y,hist):
        self.x = x
        self.y = y
        #self.pos=(x,y)
        self.hist = hist
    #description = "A knight on a 4x4 board"
    def move(self):
        import random
        offsets = [(1, 2), (1, -2), (-1, 2), (-1, -2), (2, 1), (2, -1), (-2, 1), (-2, -1)]    
        possible_pos=[]
        for move in offsets:  # try randomly chosen one
            next_x = self.x + move[0]
            next_y = self.y + move[1]
            if (next_x > 4):
                continue
            elif (next_x < 1):
                continue
            elif (next_y > 4):
                continue
            elif (next_y < 1):
                continue
            else:
                possible_pos.append((next_x, next_y)) 
        #return possible_pos
        ranmove=random.choice(possible_pos)     
        self.x=ranmove[0]
        self.y=ranmove[1]
        self.hist+=(4*(self.y-1)+self.x-1)
        #print(self.hist)
         
def move(knight):
    offsets = [(1, 2), (1, -2), (-1, 2), (-1, -2), (2, 1), (2, -1), (-2, 1), (-2, -1)]    
    possible_pos=[]
    now_pos=(knight.x,knight.y)
    for move in offsets:
        next_x = now_pos[0] + move[0]
        next_y = now_pos[1] + move[1]
        if (next_x > 4):
            continue
        elif (next_x < 1):
            continue
        elif (next_y > 4):
            continue
        elif (next_y < 1):
            continue
        else:
            possible_pos.append((next_x, next_y)) 
    return possible_pos

def move2(knight):
    deltas = [(1, 2), (1, -2), (-1, 2), (-1, -2), (2, 1), (2, -1), (-2, 1), (-2, -1)]    
    valid_position = lambda xy: all((xy[0] < 5, xy[1] < 5, xy[0] >= 1, xy[1] >= 1))
    return list(filter(valid_position, map(lambda xy: (knight.x + xy[0], knight.y + xy[1]), deltas)))

def S_sample(maxT):
    #S_Tjumps=[]
    k0=Knight(1,1,0)
    for _ in range(0,maxT):
        k0.move()
    #S_Tjumps.append(k0.hist)
    return k0.hist
    #del k0
        #print(S_Tjumps%13)

        

### Profiling, testing if parallel function is actually faster, as well as the generator form

In [ ]:
%%timeit -n1 -r1
S_sampled=Parallel(n_jobs=4)(delayed(S_sample)(16) for _ in range(1000000))  ## slower than the original list comprehension

In [49]:
%prun
S_gen=(S_sample(16) for _ in range(1000))
S_sampled2=np.array([i for i in S_gen])

53.8 ms ± 2.91 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit -n1 -r1
S_sampled=np.array([S_sample(16) for _ in range(10000000)])  
# so generator is no help 
# But 10000000 reptition, Parallel version is way faster (probably due to memory)

### Actual calculation for the challenge

In [6]:
import time

In [10]:
# T=512, use 10000 repitition
start = time.time()
S_sampled=Parallel(n_jobs=4)(delayed(S_sample)(512) for _ in range(1000))  # when over 100000 rep, almost half speed
S512=np.array(S_sampled)
S512_modulo=S512%311
divble_43=len(S512[S512%43==0])
divble_both=len(S512[(S512%43==0)&(S512%7==0)])
prob=divble_both/divble_43
print('mean of S modulo 311 = %10.11f' %S512_modulo.mean())
print('std of S modulo 311 = %10.11f' %S512_modulo.std())
print('probability that S is divisible by 7, given divisible by 43 = %10.11f' %prob)
end = time.time()
print(str(end - start)+' sec')

mean of S modulo 311 = 138.07100000000
std of S modulo 311 = 84.77838143654
probability that S is divisible by 7, given divisible by 43 = 0.16666666667
1.4923851490020752 sec


In [11]:
# T=512, use 10000 repitition
start = time.time()
S_sampled=Parallel(n_jobs=4)(delayed(S_sample)(512) for _ in range(5000))  # when over 100000 rep, almost half speed
S512=np.array(S_sampled)
S512_modulo=S512%311
divble_43=len(S512[S512%43==0])
divble_both=len(S512[(S512%43==0)&(S512%7==0)])
prob=divble_both/divble_43
print('mean of S modulo 311 = %10.11f' %S512_modulo.mean())
print('std of S modulo 311 = %10.11f' %S512_modulo.std())
print('probability that S is divisible by 7, given divisible by 43 = %10.11f' %prob)
end = time.time()
print(str(end - start)+' sec')

mean of S modulo 311 = 140.02820000000
std of S modulo 311 = 82.74959458970
probability that S is divisible by 7, given divisible by 43 = 0.12820512821
7.43947696685791 sec


In [12]:
# T=512, use 10000 repitition
start = time.time()
S_sampled=Parallel(n_jobs=4)(delayed(S_sample)(512) for _ in range(10000))  # when over 100000 rep, almost half speed
S512=np.array(S_sampled)
S512_modulo=S512%311
divble_43=len(S512[S512%43==0])
divble_both=len(S512[(S512%43==0)&(S512%7==0)])
prob=divble_both/divble_43
print('mean of S modulo 311 = %10.11f' %S512_modulo.mean())
print('std of S modulo 311 = %10.11f' %S512_modulo.std())
print('probability that S is divisible by 7, given divisible by 43 = %10.11f' %prob)
end = time.time()
print(str(end - start)+' sec')

mean of S modulo 311 = 138.95620000000
std of S modulo 311 = 82.21256158009
probability that S is divisible by 7, given divisible by 43 = 0.15068493151
14.34318494796753 sec


In [13]:
# T=512, use 10000 repitition
start = time.time()
S_sampled=Parallel(n_jobs=4)(delayed(S_sample)(512) for _ in range(100000))  # when over 100000 rep, almost half speed
S512=np.array(S_sampled)
S512_modulo=S512%311
divble_43=len(S512[S512%43==0])
divble_both=len(S512[(S512%43==0)&(S512%7==0)])
prob=divble_both/divble_43
print('mean of S modulo 311 = %10.11f' %S512_modulo.mean())
print('std of S modulo 311 = %10.11f' %S512_modulo.std())
print('probability that S is divisible by 7, given divisible by 43 = %10.11f' %prob)
end = time.time()
print(str(end - start)+' sec')

mean of S modulo 311 = 138.65673000000
std of S modulo 311 = 81.74637016839
probability that S is divisible by 7, given divisible by 43 = 0.14171572750
160.46599340438843 sec


In [14]:
start = time.time()
# T=16, use 100000 rep
S_sampled=Parallel(n_jobs=4)(delayed(S_sample)(16) for _ in range(1000000))  # when over 100000 rep, almost half speed
S16=np.array(S_sampled)
S16_modulo=S16%13
divble_13=len(S16[S16%13==0])
divble_both=len(S16[(S16%13==0)&(S16%5==0)])
prob=divble_both/divble_13
print('mean of S modulo 331 = %10.11f' %S16_modulo.mean())
print('std of S modulo 331 = %10.11f' %S16_modulo.std())
print('probability that S is divisible by 7, given divisible by 43 = %10.11f' %prob)
end = time.time()
print(str(end - start)+' sec')

mean of S modulo 331 = 5.98418600000
std of S modulo 331 = 3.73729767578
probability that S is divisible by 7, given divisible by 43 = 0.27671517939
58.27125096321106 sec


### More Prototyping as well as explicit approach confirmation for T=16

In [458]:
S16_sampled=np.array(S_Tjumps)
S16_modulo_sampled=S16_sampled%13
divble_13=len(S16_sampled[S16_sampled%13==0])
divble_both=len(S16_sampled[(S16_sampled%13==0)&(S16_sampled%5==0)])
prob=divble_both/divble_13
print('mean of S modulo 13 = %10.11f' %S16_modulo_sampled.mean())
print('std of S modulo 13 = %10.11f' %S16_modulo_sampled.std())
print('probability that S is divisible by 5, given divisible by 13 = %10.11f' %prob)

mean of S modulo 13 = 5.96158000000
std of S modulo 13 = 3.74644683715
probability that S is divisible by 5, given divisible by 13 = 0.28208673790


In [460]:
S512_sampled=np.array(S_Tjumps)
S512_modulo_sampled=S512_sampled%331
divble_43=len(S512_sampled[S512_sampled%43==0])
divble_both=len(S512_sampled[(S512_sampled%43==0)&(S512_sampled%7==0)])
prob=divble_both/divble_43
print('mean of S modulo 331 = %10.11f' %S512_modulo_sampled.mean())
print('std of S modulo 331 = %10.11f' %S512_modulo_sampled.std())
print('probability that S is divisible by 7, given divisible by 43 = %10.11f' %prob)

mean of S modulo 331 = 180.21500000000
std of S modulo 331 = 79.77349669533
probability that S is divisible by 7, given divisible by 43 = 0.21875000000


In [17]:
# T=1 >>> 2 nodes
k={}
k['k01']=Knight(1,1,[(1,1)])
k_old=deepcopy(k)
next_pos=move(k_old['k01'])
for count,pos in enumerate(next_pos):
    hist=deepcopy(k_old['k01'].hist)
    hist.append(pos)
    print(hist)
    k['k{0}'.format(str(count+1).zfill(2))]=Knight(pos[0],pos[1],hist)
print('# of nodes: '+str(len(k)))

[(1, 1), (2, 3)]
[(1, 1), (3, 2)]
# of nodes: 2


In [18]:
# T=2  >>> 8 nodes
nk=0
k_old=deepcopy(k)
for key in k_old:
    next_pos=move(k_old[key])
    for count,pos in enumerate(next_pos):
        hist=deepcopy(k_old[key].hist)
        hist.append(pos)
        #print(hist)
        k['k{0}'.format(str(count+1+nk).zfill(2))]=Knight(pos[0],pos[1],hist)
    nk=nk+len(next_pos)
print('# of nodes: '+str(len(k)))

# of nodes: 8


In [19]:
# T=3 >>> 20 nodes
nk=0
k_old=deepcopy(k)
for key in k_old:
    next_pos=move(k_old[key])
    for count,pos in enumerate(next_pos):
        hist=deepcopy(k_old[key].hist)
        hist.append(pos)
        #print(hist)
        k['k{0}'.format(str(count+1+nk).zfill(2))]=Knight(pos[0],pos[1],hist)
    nk=nk+len(next_pos)
print('# of nodes: '+str(len(k)))

# of nodes: 20


In [71]:
%%timeit # much faster but still not possible for T=512
for T in range(1,4):
    if T == 1:
        k={}
        k['k000000000001']=Knight(1,1,0)
    nk=0
    k_old=k.copy()
    for key in k_old:
        next_pos=move(k_old[key])
        for count,pos in enumerate(next_pos):
            hist=k_old[key].hist
            newsum=hist+(4*(pos[1]-1)+pos[0]-1)
            #print(newsum)
            k['k{0}'.format(str(count+1+nk).zfill(12))]=Knight(pos[0],pos[1],newsum)
        nk=nk+len(next_pos)
    #print('T = '+str(T)+' # of nodes: '+str(len(k)))

101 µs ± 6.32 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [32]:
#%%timeit # much faster but still not possible for T=512
for T in range(1,17):
    if T == 1:
        k={}
        k['k000000000001']=Knight(1,1,0)
    nk=0
    k_old=k.copy()
    for key in k_old:
        next_pos=move(k_old[key])
        for count,pos in enumerate(next_pos):
            hist=k_old[key].hist
            newsum=hist+represent(pos)
            #print(newsum)
            k['k{0}'.format(str(count+1+nk).zfill(12))]=Knight(pos[0],pos[1],newsum)
        nk=nk+len(next_pos)
    print('T = '+str(T)+' # of nodes: '+str(len(k)))

T = 1 # of nodes: 2
T = 2 # of nodes: 8
T = 3 # of nodes: 20
T = 4 # of nodes: 72
T = 5 # of nodes: 200
T = 6 # of nodes: 672
T = 7 # of nodes: 1968
T = 8 # of nodes: 6368
T = 9 # of nodes: 19168
T = 10 # of nodes: 60800
T = 11 # of nodes: 185664
T = 12 # of nodes: 582784
T = 13 # of nodes: 1793152
T = 14 # of nodes: 5597696
T = 15 # of nodes: 17292032
T = 16 # of nodes: 53825024


In [39]:
print(np.mean(aa%13))
print(np.std(aa%13))

5.98142386337
3.74082329373


In [52]:
# Question 1.5
bb=aa[aa%13==0]
cc=bb[bb%5==0]
print(len(cc)/len(bb))

0.2757314766227167


In [47]:
dd=aa[(aa%13==0) & (aa%5==0)]

In [54]:
len(dd)/len(aa)/(len(bb)/len(aa))

0.2757314766227167